## Пример на pyspark

В качестве набора данных для примера будем использовать данные конкурса про ответы студентов на тесты
https://www.kaggle.com/c/riiid-test-answer-prediction

При подключении к spark драйверу установим лимиты по памяти и по числу ядер. Также выберем номер порта для Spark UI

Нужно выбрать уникальное имя приложения и номер порта, чтобы не войти в коллизию с другими пользователями

In [1]:
%pylab inline
import pandas as pd
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("OTUS")
        .config("spark.dynamicAllocation.enabled", "true")
        .config("spark.executor.memory", "2g")
        .config("spark.driver.memory", "1g")
        .getOrCreate()
)

Данные будем читать из заранее сконвертированного parquet

In [4]:
riiid_df = spark.read.parquet("riiid/train.parquet",)

Схема данных и первые 10 записей

In [5]:
riiid_df.printSchema()

root
 |-- row_id: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- content_id: integer (nullable = true)
 |-- content_type_id: integer (nullable = true)
 |-- task_container_id: integer (nullable = true)
 |-- user_answer: integer (nullable = true)
 |-- answered_correctly: integer (nullable = true)
 |-- prior_question_elapsed_time: double (nullable = true)
 |-- prior_question_had_explanation: boolean (nullable = true)



In [6]:
riiid_df.show(10)

+--------+-----------+---------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|  row_id|  timestamp|  user_id|content_id|content_type_id|task_container_id|user_answer|answered_correctly|prior_question_elapsed_time|prior_question_had_explanation|
+--------+-----------+---------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|10059541|  953601099|218331526|      1114|              0|              650|          3|                 1|                    17000.0|                          true|
| 9882908|26098315317|214467360|       721|              0|              375|          1|                 1|                    15000.0|                          true|
|11032662|  590935189|239834028|       231|              0|              277|          1|                 1|                    18000.0|                        

In [7]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)  # to pretty print pyspark.DataFrame in jupyter
riiid_df

row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
10059541,953601099,218331526,1114,0,650,3,1,17000.0,true
9882908,26098315317,214467360,721,0,375,1,1,15000.0,true
11032662,590935189,239834028,231,0,277,1,1,18000.0,true
11487463,13117283988,249140958,7219,0,15,0,0,31750.0,false
10193731,178006028,220767027,4144,0,304,0,1,13000.0,true
11451805,1054272891,248526815,9593,0,307,0,0,14000.0,true
9751638,4175717773,212002144,8161,0,4604,2,0,44000.0,true
11102000,26201536504,241050300,8014,0,316,0,1,48000.0,true
11063074,1030127,240427856,609,0,21,1,1,18000.0,true
11747704,51839059,254609209,665,0,42,1,1,15000.0,true


Замерим время выполнения простых запросов с группировками

In [8]:
from pyspark.sql import functions as f
from pyspark.sql.functions import col

In [9]:
%%time
(
riiid_df
    .select('content_id', 'answered_correctly')
    .groupBy('content_id')
    .mean('answered_correctly')
    .show()
)

+----------+-----------------------+
|content_id|avg(answered_correctly)|
+----------+-----------------------+
|      4818|     0.4098718947459534|
|      7993|      0.438069594034797|
|      4519|     0.4880797853928695|
|      1088|     0.6364547656837357|
|      6397|     0.7980720446473871|
|      5518|     0.7800687285223368|
|      9427|      0.952957296186344|
|       148|     0.8297672832665587|
|      2366|     0.5468291250733999|
|      3749|     0.6274719401389631|
|      9465|     0.4461902348369657|
|      3794|     0.6249555634553857|
|      7240|     0.8596646072374228|
|      1580|     0.7617320584679635|
|     23336|                   -1.0|
|      6466|      0.596113567605306|
|      6654|     0.6702702702702703|
|      6620|     0.7847812401636765|
|       496|     0.7542558870632896|
|     10817|     0.7436510307738273|
+----------+-----------------------+
only showing top 20 rows

CPU times: user 6.96 ms, sys: 0 ns, total: 6.96 ms
Wall time: 13.4 s


In [10]:
%%time
(
riiid_df
    .select('user_id', 'answered_correctly')
    .where(col('answered_correctly') != -1)
    .groupby('user_id')
    .mean('answered_correctly')
    .show()
)

+---------+-----------------------+
|  user_id|avg(answered_correctly)|
+---------+-----------------------+
|252345392|       0.56480117820324|
|253500385|     0.5285296981499513|
|242039738|    0.46367041198501874|
|254408119|     0.7657534246575343|
|224426519|     0.7173333333333334|
|240485154|     0.5521978021978022|
|244175802|     0.7359550561797753|
|211646563|     0.7669491525423728|
|219176829|     0.5384615384615384|
|218611655|      0.676829268292683|
|212559006|                  0.525|
|259875659|     0.6183574879227053|
|240750690|     0.6890756302521008|
|245081718|    0.30612244897959184|
|254957588|     0.2702702702702703|
|226534187|     0.6363636363636364|
|257509511|    0.49019607843137253|
|220663840|     0.7661870503597122|
|252861630|               0.546875|
|243302977|     0.6267942583732058|
+---------+-----------------------+
only showing top 20 rows

CPU times: user 5.73 ms, sys: 989 µs, total: 6.71 ms
Wall time: 13.1 s


## Упражнение 1
Выведите top 10 студентов с наилучшими результатами. 
Обратите внимание, что поле answered_correctly равно -1, если это была лекция, а не тест. Такие записи нужно исключить.

## Упражнение 2
Выведите top 10 задач с наихудшими результатами

## pyspark user defined functions (UDF)

Как и для других языков, поддерживаемых Spark, для python есть возможность использовать UDF. При этом возникают дополнительные накладные расходы по сравнению с Java и Scala на маршалинг данных.

In [ ]:
from pyspark.sql.types import LongType

def to_months(ms):
    return ms // 31536000000 // 12 #1 year = 31536000000 ms

to_months_udf = f.udf(to_months, LongType())

Замерим время выполнения без UDF

In [ ]:
%%time
(
    riiid_df
        .select("content_id", "timestamp")
        .groupby("content_id")
        .mean("timestamp")
        .show()
)

Применим простой UDF к похожему запросу

In [ ]:
%%time
(
    riiid_df
        .select("content_id", to_days_udf("timestamp").alias("months"))
        .groupBy("content_id")
        .mean("months")
        .show()
)

Перепишем логику, которая была в UDF

In [ ]:
%%time
(
 riiid_df
    .select("content_id", (col("timestamp") / 31536000000 / 12).alias("months"))
    .groupby("content_id")
    .mean("months")
    .show()
)

## Упражнение 4
Постройте гистограмму по числу месяцев до первого взаимодействия студента с заданием

## Обогащение данных

Таблица с вопросами лежит в отдельном файле questions.csv. 

In [ ]:
questions = spark.read.csv("riiid/questions.csv", header=True, inferSchema=True)

In [ ]:
questions.count()

In [ ]:
questions.printSchema()

Объединим ее с ответами при условии, что эта запись ссылкается на вопрос если content_type_id и content_id - идентификатор вопроса.

In [ ]:
df = (
    riiid_df.
        where(f.col("content_type_id") == 0).
        join(questions, riiid_df.content_id == questions.question_id, 'left')
)

In [ ]:
df

Проверим, что вероястность правильного ответа не зависит от его номера.

In [ ]:
%%time
(
df
    .select('correct_answer', 'answered_correctly')
    .groupby('correct_answer')
    .mean('answered_correctly')
    .show()
)

## Упражнение 5

В файле "riiid/lectures.csv" хранится информация об лекциях. Объедините эту таблицу с основным набором данных при условии, что content_type_id == 1.